In [1]:
import geopandas as gpd
import pandas as pd
from shapely import wkt
from osgeo import gdal, osr, ogr  # necessario intalar a whell manualmente https://www.lfd.uci.edu/~gohlke/pythonlibs/
import math
import time
import pyproj
#import rasterio
#from rasterio import features

In [20]:
file = 'gpd.csv'

MERCATOR = pyproj.CRS('epsg:3857')
WGS84 = pyproj.CRS('epsg:4326')
SAD1969  = pyproj.CRS('esri:102033')

df = pd.read_csv(file)


df['geometry'] = df['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(df, crs='epsg:4326')

arq = gdal.Open('img.shp')


wkt_geom = [i.wkt for i in gdf.geometry]
#gdf.explore()

In [ ]:
dfn = pd.DataFrame(data = [gdf.geometry.x]

In [27]:
ds = gdal.Open('rasterize.tif')

In [35]:
ds.GetRasterBand(1).ReadAsArray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [49]:
x_size = gdf.total_bounds[2]-gdf.total_bounds[0]
y_size = gdf.total_bounds[3]-gdf.total_bounds[1]
x_0 = gdf.total_bounds[0]
y_0 = gdf.total_bounds[3]

 

pix_size = 0.000002
pixels_in_x = math.ceil(x_size/pix_size)
pixels_in_y = math.ceil(y_size/pix_size)

 
sr_wkt = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'
sr = osr.SpatialReference( sr_wkt )


# Create a memory raster to rasterize into.

 

target_ds = gdal.GetDriverByName('MEM').Create('', pixels_in_x, pixels_in_y, 4, gdal.GDT_Byte)
target_ds.SetGeoTransform((x_0, pix_size, 0, y_0, 0, -pix_size))

 

# Create a memory layer to rasterize from.

 
rast_ogr_ds  = ogr.GetDriverByName('Memory').CreateDataSource('wrk')
rast_mem_lyr = rast_ogr_ds.CreateLayer('poly', srs=sr)


for g in gdf[gdf['cor'] == 'produzindo_rastro'].geometry:
    feat = ogr.Feature(rast_mem_lyr.GetLayerDefn())
    feat.SetGeometryDirectly(ogr.Geometry(wkt = g.wkt))
    rast_mem_lyr.CreateFeature(feat)

gdal.RasterizeLayer(target_ds, [1,2,3,4], rast_mem_lyr)

""" , options = ["BURN_VALUE_FROM="]   ,  ,options=['ATTRIBUTE=values'] """

proj = 'PROJCS["unnamed",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["Degree",0.0174532925199433]],PROJECTION["EPSG:4326"],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","4326"]]'
target_ds.SetProjection(proj)

 

tiffdriver = gdal.GetDriverByName('GTiff')
tiffdriver.CreateCopy('rasterize.tif', target_ds)

 

pngdriver = gdal.GetDriverByName('PNG')
pngdriver.CreateCopy('rasterize.png', target_ds)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x0000023C1C86C990> >